In [14]:
%load_ext autoreload
%autoreload 2

from hyper_qa.model import HyperQA
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
tf.random.set_random_seed(212)

In [3]:
embedding_matrix = np.random.normal(0, 1, (1000, 300))
def generate_samples(max_length, sample_size, vocab_size=1000, pad=False):
    lengths = []
    sample_sequences = []
    for i in range(sample_size):
#         text = np.random.randint(1,1000, np.random.randint(1,max_length))
        text = np.arange(i%max_length)
        if pad:
            sample = np.concatenate((text, [0] * (max_length - len(text))))
        else:
            sample = text
        lengths.append(len(text))
        sample_sequences.append(sample)
    if pad:
        sample_sequences = np.vstack(sample_sequences)
    return sample_sequences

In [4]:
max_length = 50
q = generate_samples(max_length, 200, pad=True)
a = generate_samples(max_length, 200, pad=True)
b = generate_samples(max_length, 200, pad=True)

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((q, a, b))
dataset = dataset.shuffle(200).batch(1)
data_iter = dataset.make_one_shot_iterator()

Instructions for updating:
Colocations handled automatically by placer.


In [19]:

model = HyperQA(1000, max_length, embedding_matrix=embedding_matrix)

In [20]:
q, a, b = data_iter.get_next()
logits = model((q,a,b), training=True)

InvalidArgumentError: Invalid reduction dimension (1 for input with 1 dimension(s) [Op:Sum]